베이즈 네트워크의 근사 추론(approximate inference). 여기 제공하는 코드는 GitHub aima-python의 코드를 기반으로 일부 수정한 것임.

In [12]:
# 베이즈 네트워크 기본 코드를 bayesnet.py에 저장해뒀음
from bayesnet import *
from probability import *
from functools import reduce
import random
seed = 23

## 정확추론

In [13]:
# 열거
def enumeration_ask(X, e, bn):
    """베이즈 네트워크 bn에 대해, 증거 e(딕셔너리)가 주어졌을 때 변수 X의 조건부 확률을 리턴함."""
    assert X not in e, "Query variable must be distinct from evidence"
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        Q[xi] = enumerate_all(bn.variables, extend(e, X, xi), bn)
    return Q.normalize()


def enumerate_all(variables, e, bn):
    """증거 e와 일관된 P(variables | e{others})의 합을 리턴함.
    P: 베이즈 네트워크 bn으로 표현되는 결합 확률 분포,
    e{others}: bn에서 variables를 제외한 다른 변수들로 제한된 증거.
    variables에서 부모는 자식들보다 앞에 위치해야 함."""
    if not variables:
        return 1.0
    Y, rest = variables[0], variables[1:]
    Ynode = bn.variable_node(Y)
    if Y in e:
        return Ynode.p(e[Y], e) * enumerate_all(rest, e, bn)
    else:
        return sum(Ynode.p(y, e) * enumerate_all(rest, extend(e, Y, y), bn)
                   for y in bn.variable_values(Y))
    
    
# 변수 소거 알고리즘
def elimination_ask(X, e, bn):
    """변수 소거를 통해 베이즈 네트워크 bn의 P(X|e)를 계산함. e: 딕셔너리"""
    assert X not in e, "Query variable must be distinct from evidence"
    factors = []
    for var in reversed(bn.variables):
        factors.append(make_factor(var, e, bn))
        if is_hidden(var, X, e):
            factors = sum_out(var, factors, bn)
    return pointwise_product(factors, bn).normalize()


def is_hidden(var, X, e):
    """P(X|e)를 질의할 때 var이 미관측 변수인가?"""
    return var != X and var not in e


def make_factor(var, e, bn):
    """e가 주어졌을 때 bn의 결합 확률 분포에서 var에 대한 인수를 리턴함.
    bn의 완전 결합 확률 분포는 bn 변수들에 대한 이 인수들의 성분별 곱(pointwise product)임."""
    node = bn.variable_node(var)
    variables = [X for X in [var] + node.parents if X not in e]
    cpt = {event_values(e1, variables): node.p(e1[var], e1)
           for e1 in all_events(variables, bn, e)}
    return Factor(variables, cpt)


def pointwise_product(factors, bn):
    return reduce(lambda f, g: f.pointwise_product(g, bn), factors)


def sum_out(var, factors, bn):
    """값들에 대해 합산함으로써 모든 인수들에서 var를 삭제함."""
    result, var_factors = [], []
    for f in factors:
        (var_factors if var in f.variables else result).append(f)
    result.append(pointwise_product(var_factors, bn).sum_out(var, bn))
    return result


class Factor:
    """결합 확률 분포의 한 인수"""

    def __init__(self, variables, cpt):
        self.variables = variables
        self.cpt = cpt

    def pointwise_product(self, other, bn):
        """두 인수의 성분별 곱 계산. 변수들은 합집합이 됨."""
        variables = list(set(self.variables) | set(other.variables))
        cpt = {event_values(e, variables): self.p(e) * other.p(e) for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def sum_out(self, var, bn):
        """값들에 대해 합산함으로써 var를 삭제하여 인수를 생성함."""
        variables = [X for X in self.variables if X != var]
        cpt = {event_values(e, variables): sum(self.p(extend(e, var, val)) for val in bn.variable_values(var))
               for e in all_events(variables, bn, {})}
        return Factor(variables, cpt)

    def normalize(self):
        """확률값 리턴. 하나의 변수여야 함."""
        assert len(self.variables) == 1
        return ProbDist(self.variables[0], {k: v for ((k,), v) in self.cpt.items()})

    def p(self, e):
        """e에 대한 조건부 확률표 조회"""
        return self.cpt[event_values(e, self.variables)]


def all_events(variables, bn, e):
    """모든 변수들에 대한 값으로 e를 확장하여 yield."""
    if not variables:
        yield e
    else:
        X, rest = variables[0], variables[1:]
        for e1 in all_events(rest, bn, e):
            for x in bn.variable_values(X):
                yield extend(e1, X, x)


def extend(s, var, val):
    """딕셔너리 s를 복사하고 var에 값 val을 세팅하여 확장하여 그 복사본을 리턴함."""
    return {**s, var: val}

## 근사 추론 알고리즘 구현

In [14]:
# 무증거 랜덤 샘플링
def prior_sample(bn):
    """
    베이즈 네트워크 bn의 완전 결합 확률 분포에서 무작위 샘플링.
    {변수: 값} 형식의 딕셔너리가 리턴됨.
    """
    event = {}
    for node in bn.nodes:
        event[node.variable] = node.sample(event)
    return event

In [3]:
# 기각 샘플링
def rejection_sampling(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    N개 샘플이 모두 기각되면 ZeroDivisionError 발생.
    """
    counts = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample = prior_sample(bn)
        if consistent_with(sample, e):
            counts[sample[X]] += 1
    return ProbDist(X, counts)


def consistent_with(event, evidence):
    """event가 evidence와 일관되는가?(증거와 부합하는가?)"""
    return all(evidence.get(k, v) == v for k, v in event.items())

In [4]:
# 가능도 가중치
def likelihood_weighting(X, e, bn, N=10000):
    """
    N개의 샘플을 사용하여 P(X|e)를 추정함.
    """
    W = {x: 0 for x in bn.variable_values(X)}
    for j in range(N):
        sample, weight = weighted_sample(bn, e)
        W[sample[X]] += weight
    return ProbDist(X, W)


def weighted_sample(bn, e):
    """
    증거 e와 일관된 이벤트를 bn으로부터 샘플링함.
    이벤트와 가중치(이벤트가 증거에 부합할 가능도)를 리턴함.
    """
    w = 1
    event = dict(e)
    for node in bn.nodes:
        Xi = node.variable
        if Xi in e:
            w *= node.p(e[Xi], event)
        else:
            event[Xi] = node.sample(event)
    return event, w


In [5]:
# Gibbs 샘플링
def gibbs_ask(X, e, bn, N=1000):
    """Gibbs 샘플링."""
    assert X not in e, "Query variable must be distinct from evidence"
    counts = {x: 0 for x in bn.variable_values(X)}
    Z = [var for var in bn.variables if var not in e]
    state = dict(e)
    for Zi in Z:
        state[Zi] = random.choice(bn.variable_values(Zi))
    for j in range(N):
        for Zi in Z:
            state[Zi] = markov_blanket_sample(Zi, state, bn)
            counts[state[X]] += 1
    return ProbDist(X, counts)


def markov_blanket_sample(X, e, bn):
    """
    P(X | mb)에 따라 샘플을 리턴함.
    mb: X의 마르코프 블랭킷에 속한 변수들. 그 값은 이벤트 e에서 취함.
    x의 마르코프 블랭킷: X의 부모들, 자식들, 자식들의 부모들
    """
    Xnode = bn.variable_node(X)
    Q = ProbDist(X)
    for xi in bn.variable_values(X):
        ei = extend(e, X, xi)
        Q[xi] = Xnode.p(xi, e) * product(Yj.p(ei[Yj.variable], ei) for Yj in Xnode.children)
    # 불리언 변수로 가정함.
    return probability(Q.normalize()[True])


def product(numbers):
    """numbers에 있는 값들을 모두 곱한 결과를 리턴함. 예: product([2, 3, 10]) == 60"""
    result = 1
    for x in numbers:
        result *= x
    return result

## 근사 추론 예

In [6]:
# 결정론적 노드 o
Aplus = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('정시기상', '체력좋음', {T: 0.99, F: 0.90}),
    ('정시출석','정시기상',{T:1, F:0}),
    ('에이플', '족보보유 사전지식보유 자료3회독 정시출석', {(T, T, T, T): 0.97, (T, T, F, T): 0.7, (T, F, T, T): 0.85, (F, T, T, T): 0.7, (T, F, F, T): 0.5, (F, T, F, T): 0.3, (F, F, T, T): 0.4, (F, F, F, T): 0.05, (T, T, T, F): 0.5, (T, T, F, F): 0.4, (T, F, T, F): 0.35, (F, T, T, F): 0.35, (T, F, F, F): 0.2, (F, T, F, F): 0.15, (F, F, T, F): 0.1, (F, F, F, F): 0.03})
])

In [16]:

Aplus3 = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('정시출석', '체력좋음', {T: 0.99, F: 0.90}),
    ('에이플', '족보보유 사전지식보유 자료3회독 정시출석', {(T, T, T, T): 0.97, (T, T, F, T): 0.7, (T, F, T, T): 0.85, (F, T, T, T): 0.7, (T, F, F, T): 0.5, (F, T, F, T): 0.3, (F, F, T, T): 0.4, (F, F, F, T): 0.05, (T, T, T, F): 0.5, (T, T, F, F): 0.4, (T, F, T, F): 0.35, (F, T, T, F): 0.35, (T, F, F, F): 0.2, (F, T, F, F): 0.15, (F, F, T, F): 0.1, (F, F, F, F): 0.03})
])

In [17]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus3, 10000)
    print(dist.show_approx())
    print(dist[True])

False: 0.649, True: 0.351
0.3509909090909091
False: 0.65, True: 0.35
0.3502090909090909
False: 0.655, True: 0.345
0.34450909090909093
False: 0.662, True: 0.338
0.3382909090909091
False: 0.654, True: 0.346
0.3461909090909091
False: 0.667, True: 0.333
0.3326909090909091
False: 0.661, True: 0.339
0.3385
False: 0.645, True: 0.355
0.3547909090909091
False: 0.633, True: 0.367
0.3668
False: 0.653, True: 0.347
0.34650909090909093


In [22]:
N = 1000
all_observations = [prior_sample(Aplus) for x in range(N)]
all_observations[:1]

[{'교수님의수제자': False,
  '선배와친분': False,
  '족보보유': False,
  '외부대외활동': False,
  '높은아이큐': True,
  '사전지식보유': False,
  '체력좋음': False,
  '매주예복습': False,
  '자료3회독': False,
  '정시기상': True,
  '정시출석': True,
  '에이플': True}]

In [23]:
# P(에이플=True)
Aplus_true = [observation for observation in all_observations if observation['에이플'] == True]
answer = len(Aplus_true) / N
print(answer)

0.323


In [24]:
# 다시 샘플링하여 계산
N = 1000
all_observations = [prior_sample(Aplus) for x in range(N)]
Aplus_true = [observation for observation in all_observations if observation['정시출석'] == True]
answer = len(Aplus_true) / N
print(answer)

0.918


In [ ]:
# P(교수님의수제자=True | 에이플=True)
pp_and_Aplus = [observation for observation in Aplus_true if observation['교수님의수제자'] == True]
answer = len(pp_and_Aplus) / len(Aplus_true)
print(answer)

In [ ]:
# 기각 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
#random.seed(seed)
dist = rejection_sampling('족보보유', dict(에이플=True, 높은아이큐=True), Aplus, 10000)
print(dist.show_approx())
print(dist[True])

In [ ]:
weighted_sample(Aplus, dict(에이플=True))

In [ ]:
# 가능도 가중치를 사용하여 P(에이플=True | 교수님의수제자=True) 추정
#random.seed(seed)
dist = likelihood_weighting('에이플', dict(출석=True), Aplus, 1000)
print(dist.show_approx())
print(dist[True])

In [ ]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus, 10000)
    print(dist.show_approx())
    print(dist[True])

In [ ]:
# 결정론적인 노드 x
Aplus2 = BayesNet([
    ("교수님의수제자", '', 0.05),
    ('선배와친분', "교수님의수제자", {T: 0.8, F: 0.3}),
    ('족보보유', '선배와친분', {T: 0.7, F: 0.1}),
    ('외부대외활동', '교수님의수제자', {T: 0.85, F: 0.1}),
    ('높은아이큐', '', 0.2),
    ('사전지식보유', '외부대외활동 높은아이큐', {(T, T): 0.9, (T, F): 0.75, (F, T): 0.35, (F, F): 0.05}),
    ('체력좋음', '', 0.25),
    ('매주예복습', '체력좋음', {T: 0.6, F: 0.2}),
    ('자료3회독', '매주예복습', {T: 0.9, F: 0.15}),
    ('에이플', '족보보유 사전지식보유 자료3회독', {(T, T, T): 0.97, (T, T, F): 0.7, (T, F, T): 0.85, (F, T, T): 0.7, (T, F, F): 0.5, (F, T, F): 0.3, (F, F, T): 0.4, (F, F, F): 0.05})
])

In [ ]:
# Gibbs 샘플링을 사용하여 P(에이플=True | 교수님의수제자=True) 추정 안고장남
# 결정론 : .335, 비결정론 : .362
for _ in range(10):
    dist = gibbs_ask('에이플', dict(), Aplus2, 10000)
    print(dist.show_approx())
    print(dist[True])

In [15]:
ans_dist = elimination_ask('에이플', dict(교수님의수제자=True), Aplus)
ans_dist.show_approx()

'False: 0.436, True: 0.564'